In [24]:
"""
This is our 2-stage testing script (binary and multiclass)

Author: Wesley
"""
import numpy as np
import pandas as pd
import pickle
import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

Load dataset and get binary and multiclass y columns.

In [5]:
binary_train = pd.read_csv("binary_train.csv")
binary_test = pd.read_csv("binary_test.csv")

multiclass_train = pd.read_csv("multiclass_train.csv")
multiclass_test = pd.read_csv("multiclass_test.csv")

print(binary_train[" Label"].value_counts())
print(binary_test[" Label"].value_counts())
print(multiclass_train[" Label"].value_counts())
print(multiclass_test[" Label"].value_counts())

BENIGN     79997
TFTP        6667
Syn         6667
UDP         6667
DNS         6667
UDP-lag     6666
LDAP        6666
Portmap     6666
SSDP        6666
NTP         6666
NetBIOS     6666
MSSQL       6666
SNMP        6666
Name:  Label, dtype: int64
BENIGN     19999
SSDP        1667
NetBIOS     1667
NTP         1667
LDAP        1667
UDP-lag     1667
SNMP        1667
MSSQL       1667
Portmap     1667
UDP         1666
TFTP        1666
Syn         1666
DNS         1666
Name:  Label, dtype: int64
LDAP       13333
DNS        13333
Portmap    13333
UDP        13333
MSSQL      13333
UDP-lag    13333
Syn        13333
NetBIOS    13333
TFTP       13333
SSDP       13332
NTP        13332
SNMP       13332
Name:  Label, dtype: int64
SNMP       3334
SSDP       3334
NTP        3334
Portmap    3333
NetBIOS    3333
UDP-lag    3333
MSSQL      3333
LDAP       3333
DNS        3333
UDP        3333
TFTP       3333
Syn        3333
Name:  Label, dtype: int64


Split and encode data.

In [8]:
bin_y_train = binary_train[" Label"].copy()
bin_x_train = binary_train.drop([" Label"], axis=1)

bin_y_test = binary_test[" Label"].copy()
bin_x_test = binary_test.drop([" Label"], axis=1)

bin_y_train = [0 if x=="BENIGN" else 1 for x in bin_y_train.values]
bin_y_test = [0 if x=="BENIGN" else 1 for x in bin_y_test.values]

print("The text labels from the encoding will be passed to classification report so we can interpret our results more easily.\n")
binary_labels = ["BENIGN", "ATTACK"]

print("Binary Label Encodings (in order of digits 0 -> 1): ")
print(binary_labels)

multi_y_train = multiclass_train[" Label"].copy()
multi_x_train = multiclass_train.drop([" Label"], axis=1)

multi_y_test = multiclass_test[" Label"].copy()
multi_x_test = multiclass_test.drop([" Label"], axis=1)

# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()

multi_y_train = le.fit_transform(multi_y_train.values)
multi_y_test = le.transform(multi_y_test.values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multi_y_test))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)

The text labels from the encoding will be passed to classification report so we can interpret our results more easily.

Binary Label Encodings (in order of digits 0 -> 1): 
['BENIGN', 'ATTACK']

Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load models

In [25]:
# Load a binary model
#stage_1 = pickle.load(open("rf_binary_test.pickle", 'rb'))
stage_1 = keras.models.load_model('CNN_binary_final.h5')
# Load a multiclass model
stage_2 = KNeighborsClassifier(algorithm = 'kd_tree', n_neighbors=47, weights = 'distance')

Stage 1 Prediction (Binary)

In [19]:
scal = StandardScaler()
scal = scal.fit(bin_x_train)
bin_x_test_reshaped = scal.transform(bin_x_test)
y_pred = stage_1.predict(bin_x_test_reshaped)
y_pred = [1 if x>=0.5 else 0 for x in y_pred]

1250/1250 [==============================] - 2s 2ms/step


In [20]:
print(classification_report(bin_y_test, y_pred, digits=6, target_names=binary_labels))

              precision    recall  f1-score   support

      BENIGN   0.999650  0.999900  0.999775     19999
      ATTACK   0.999900  0.999650  0.999775     20000

    accuracy                       0.999775     39999
   macro avg   0.999775  0.999775  0.999775     39999
weighted avg   0.999775  0.999775  0.999775     39999



Housekeeping to prepare the dataset for 2nd stage.

In [58]:
# Create new dataframe out of binary predictions and multiclass labels that can be used to compare them later.
X_agg = bin_x_test.copy()
X_agg["binary_pred"] = y_pred.copy()
X_agg[" Label"] = binary_test[" Label"].copy()
#X_agg[" Label"] = le.transform(X_agg[" Label"].values)
X_agg.loc[~X_agg[" Label"].isin(le.classes_)," Label"] = -1 
X_agg.loc[X_agg[" Label"].isin(le.classes_)," Label"] = le.transform(X_agg[" Label"][X_agg[" Label"].isin(le.classes_)])

# Slice out the portions that we've identified as benign and replace their label with that of the multiclass benign label for comparison later.
X_rem = X_agg[X_agg["binary_pred"] == 0]
X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)

# Slice out the portions we've labeled as attack for 2nd stage classification.
X_mult = X_agg[X_agg["binary_pred"] == 1]
X_mult = X_mult.drop(["binary_pred"], axis = 1)
y_mult = X_mult[" Label"].values.copy()

X_mult = X_mult.drop([" Label"], axis = 1)

C:\Users\icarus\AppData\Local\Temp\ipykernel_33908\2310080828.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)


2nd stage prediction

In [59]:
# Some reshaping is needed for the LSTM model to work
stage_2.fit(multi_x_train, multi_y_train)

y_pred2 = stage_2.predict(X_mult)
#y_pred2 = [np.argmax(x) for x in y_pred2]

In [60]:
print(np.unique(y_pred2, return_counts=True))
print(np.unique(list(y_mult), return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([1757, 1266, 1658, 1679, 1060, 2329, 1955, 1606, 1028, 1662, 1781,
       2214], dtype=int64))
(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([   2, 1665, 1667, 1667, 1664, 1667, 1666, 1667, 1666, 1666, 1665,
       1666, 1667], dtype=int64))


In [61]:
print(multiclass_labels)
multiclass_labels_add = ["BENIGN"]
multiclass_labels_add.extend(multiclass_labels)
print(multiclass_labels_add)

['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']
['BENIGN', 'DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


In [65]:
print(classification_report(list(y_mult), y_pred2, digits=6, target_names=multiclass_labels_add))

              precision    recall  f1-score   support

      BENIGN   0.000000  0.000000  0.000000         2
         DNS   0.648264  0.684084  0.665693      1665
        LDAP   0.671406  0.509898  0.579611      1667
       MSSQL   0.954162  0.949010  0.951579      1667
         NTP   0.977963  0.986779  0.982351      1664
     NetBIOS   0.749057  0.476305  0.582325      1667
     Portmap   0.619579  0.866146  0.722403      1666
        SNMP   0.722762  0.847630  0.780232      1667
        SSDP   0.724159  0.698079  0.710880      1666
         Syn   0.956226  0.590036  0.729770      1666
        TFTP   0.994585  0.992793  0.993688      1665
         UDP   0.710275  0.759304  0.733972      1666
     UDP-lag   0.674797  0.896221  0.769905      1667

    accuracy                       0.771243     19995
   macro avg   0.723326  0.712022  0.707878     19995
weighted avg   0.783494  0.771243  0.766754     19995



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Final Calculation of Results

In [67]:
y_pred_final = list(X_rem["binary_pred"].values.copy())
y_pred_final.extend(y_pred2)

y_true_final = list(X_rem[" Label"].values.copy())
y_true_final.extend(list(y_mult))

print(classification_report(y_true_final, y_pred_final, digits=6, target_names=multiclass_labels_add))

              precision    recall  f1-score   support

      BENIGN   0.999650  0.999900  0.999775     19999
         DNS   0.648264  0.683673  0.665498      1666
        LDAP   0.671406  0.509898  0.579611      1667
       MSSQL   0.954162  0.949010  0.951579      1667
         NTP   0.977963  0.985003  0.981470      1667
     NetBIOS   0.749057  0.476305  0.582325      1667
     Portmap   0.619579  0.865627  0.722222      1667
        SNMP   0.722762  0.847630  0.780232      1667
        SSDP   0.724159  0.697660  0.710663      1667
         Syn   0.956226  0.590036  0.729770      1666
        TFTP   0.994585  0.992197  0.993389      1666
         UDP   0.710275  0.759304  0.733972      1666
     UDP-lag   0.674797  0.896221  0.769905      1667

    accuracy                       0.885472     39999
   macro avg   0.800222  0.788651  0.784647     39999
weighted avg   0.891619  0.885472  0.883243     39999

